In [17]:
## imports
import sys
import os
import numpy as np
import xarray as xr
import cmocean
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from matplotlib import gridspec
import cartopy
import cartopy.crs as ccrs
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import eofs
from eofs.xarray import Eof
import cftime
import nc_time_axis

import mtspec

# from pylab import *

import datetime
import time
import glob, os
import math
import matplotlib.colors as colors
import xesmf as xe
from defs import regularAtlmask

In [1]:
datafolder = ''

In [18]:
experiment = 'E280' 
model = 'CCSM4'

if experiment == 'E280':
    preamble = 'b40.B1850.f09_g16.preind.pop.h.'
    preamble2 = '.0081-0180.annavg.nc'
else:
    preamble = 'b40.B1850.f09_g16.PMIP4-pliomip2.pop.h.'
    preamble2 = '.1001-1100.annavg.nc'

a = 'b40.B1850.f09_g16.PMIP4-pliomip2.pop.h.MOC.1001-1100.ann.nc'
folder = datafolder+'Raw_data/CCSM4/'

def filename(variable_string):
    filename = folder+experiment+'/'+preamble+variable_string+preamble2
    
    return filename

In [19]:
dsV = xr.open_dataset(filename('VVEL'))
dsT = xr.open_dataset(filename('TEMP'))
dsS = xr.open_dataset(filename('SALT'))
dsVNT = xr.open_dataset(filename('VNT'))
dsOHT = xr.open_dataset(filename('N_HEAT'))
dsgrid = xr.open_dataset(folder+experiment+'/'+experiment+'.SST.mean_month.nc', decode_times=False)

In [20]:
if experiment == 'E280':
    dsT = dsT.drop(['ULAT','ULONG','z_t_150m','z_w','z_w_top','z_w_bot','lat_aux_grid','moc_z'])
    dsV = dsV.drop(['TLAT','TLONG','z_t_150m','z_w','z_w_top','z_w_bot','lat_aux_grid','moc_z'])
    dsS = dsS.drop(['ULAT','ULONG','lat_aux_grid'])

In [21]:
regridder = xe.Regridder(dsS, dsgrid, 'bilinear', periodic = True, ignore_degenerate=True)

var = regridder(dsS.SALT[0])

ds = xr.Dataset(data_vars = dict(so=(("z","lat","lon"), var)), coords = dict(z = var.z_t.values/100, lat = var.latitude.values, lon = var.longitude.values))
xr.Dataset.to_netcdf(ds, path = datafolder+'Processed/'+model+'/'+experiment+'/so_1x1_100yr.nc')

In [22]:
regridder = xe.Regridder(dsV, dsgrid, 'bilinear', periodic = True, ignore_degenerate=True)

var = regridder(dsV.VVEL[0])

ds = xr.Dataset(data_vars = dict(vo=(("z","lat","lon"), var)), coords = dict(z = var.z_t.values/100, lat = var.latitude.values, lon = var.longitude.values))
xr.Dataset.to_netcdf(ds, path = datafolder+'Processed/'+model+'/'+experiment+'/vo_1x1_100yr.nc')

In [23]:
regridder = xe.Regridder(dsT, dsgrid, 'bilinear', periodic = True, ignore_degenerate=True)

var = regridder(dsT.TEMP[0])

ds = xr.Dataset(data_vars = dict(thetao=(("z","lat","lon"), var)), coords = dict(z = var.z_t.values/100, lat = var.latitude.values, lon = var.longitude.values))
xr.Dataset.to_netcdf(ds, path = datafolder+'/Processed/'+model+'/'+experiment+'/thetao_1x1_100yr.nc')